In [61]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from PIL import Image

In [ ]:
L = 10
T = 50

dx = 1e-2
dt = 5e-3

x = np.arange(0, L+dx, dx)
t = np.arange(0, T+dt, dt)

Nt, Nx = len(t), len(x)

def V(x):
    "Potential function"
    return 1 * (x - L/2)**2

def u_0(x):
    "Initial Condition"
    return np.exp(-200*(x - L/4)**2)

def D2u(n):
    D2 = np.zeros((n, n))
    for i in range(1, n-1):
        D2[i, [i-1, i, i+1]] = [1, -2, 1]
    D2[0, [0, 1]]    = [-2, 1]
    D2[-1, [-2, -1]] = [1, -2]
    return 1/dx * D2


I  = np.diag(np.ones(Nx))
u  = np.zeros((Nt, Nx), dtype=complex)

a = 1e0
def H(ti):
    return a*1j*D2u(Nx) + -1j*I*V(x)

for i, ti in enumerate(t):
    if i == 0:
        u[0, :] = u_0(x)
    else:
        j  = i-1
        uj = u[j, :]
        tj = t[j]
        u[i, :] = np.linalg.solve((I - dt/2 * H(tj)), (I + dt/2 * H(ti))@uj)
    
    if np.max(np.abs(u[i, :])) > 1e2:
        print(f"Blow up at time {ti:.3f}s")
        break
    
    if i % 100 == 0:
        print(f"Time {ti:.3f}s, max(u) = {np.max(np.abs(u[i, :])):.3f}")
        



Time 0.000s, max(u) = 1.000


In [ ]:
def save_gif_PIL(outfile, files, fps=5, loop=0):
    "Helper function for saving GIFs"
    imgs = [Image.open(file) for file in files]
    imgs[0].save(fp=outfile, format='GIF', append_images=imgs[1:], save_all=True, duration=round(1000/fps), loop=loop)

def make_plot(save=False, file_name=f"PDE_solution.png", ti=0):
    "Helper function for plotting"
    fig, ax = plt.subplots()
    ax.plot(x, V(x), color="gray", lw=1, label="Potential", ls="--")
    ax.plot(x, np.abs(u[i, :]), color="black", lw=2)
    ax.plot(x, np.real(u[i, :]), color="Red", lw=1)
    ax.plot(x, np.imag(u[i, :]), color="Blue", lw=1)
    ax.set_xlim(0, L)
    ax.set_ylim(-1.05, 1.05)
    ax.grid()
    ax.set_title(f"Time = {ti:.3f}s")
    if save:
        fig.savefig(file_name, bbox_inches="tight", pad_inches=0.1, dpi=100, facecolor="white")

In [ ]:
import os
import shutil

folder_path="plots"
for item in os.listdir(folder_path):
    item_path = os.path.join(folder_path, item)
    try:
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)  # Remove file or symbolic link
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)  # Remove subdirectory and its contents
    except Exception as e:
        print(f"Failed to delete {item_path}. Reason: {e}")

In [ ]:
files = []
fps   = 20
frame_indicies = np.linspace(0, Nt-1, int(fps * 10), dtype=int)
for i in frame_indicies:
    ti = t[i]
    file = f"plots/PDE_sol_{ti:.5f}.png"
    make_plot(save=True, file_name=file, ti=ti)
    files.append(file)
    plt.close("all")
    
save_gif_PIL("PDE_solution.gif", files, fps=fps, loop=0)